# Python Functional Programming — map(), filter(), reduce() (Tricky Questions + Practice)

This notebook focuses on **map / filter / reduce** with **lots of examples**, common interview-style **tricky questions**, and pitfalls.

## What you will learn
- What map/filter/reduce return (iterators!)
- Lambda vs def in these functions
- Multiple iterables in `map`
- Filtering with multiple conditions
- Reduce with initializer (important!)
- Reduce on empty lists (tricky)
- reduce for max/min, product, string join, dict merge, frequency maps
- Real-world data-engineering flavored examples


## 0) Setup

`reduce` lives in `functools`.


In [ ]:
from functools import reduce


## 1) Key Concept: map/filter return iterators (TRICKY!)

**Trick question:** Why does printing a `map` show something like `<map object at ...>`?

Because `map()` returns a lazy **iterator**. You usually convert it to `list()`.

**Another trick:** Once you iterate over it, it’s exhausted.


In [ ]:
nums = [1, 2, 3]
m = map(lambda x: x * 2, nums)
print("m =", m)

print("First consumption:", list(m))
print("Second consumption (empty):", list(m))  # iterator is exhausted


## 2) map() — Basics

### Pattern
```python
map(function, iterable)
```

map applies `function` to **each element**.


In [ ]:
nums = [1, 2, 3, 4]
squares = list(map(lambda x: x * x, nums))
print("squares:", squares)


### map with a named function (often clearer than lambda)


In [ ]:
def normalize(s: str) -> str:
    return s.strip().lower()

raw = ["  Order ", "PRODUCTS ", " customers"]
clean = list(map(normalize, raw))
print("clean:", clean)


## 3) map() — TRICKY: multiple iterables

### Pattern
```python
map(func, iterable1, iterable2, ...)
```

**Trick:** `map` stops at the **shortest** iterable.


In [ ]:
a = [1, 2, 3, 4]
b = [10, 20]

pair_sums = list(map(lambda x, y: x + y, a, b))
print("pair_sums (stops at shortest):", pair_sums)


### map TRICKY: converting input strings into numbers

Common HackerRank pattern:
```python
values = list(map(int, input().split()))
```

Here’s a simulated version:


In [ ]:
line = "10  20  30"
values = list(map(int, line.split()))
print(values)


## 4) filter() — Basics

### Pattern
```python
filter(predicate_function, iterable)
```

- predicate must return True/False
- filter keeps items where predicate is True
- filter returns an iterator too


In [ ]:
nums = [1, 2, 3, 4, 5, 6]
evens = list(filter(lambda x: x % 2 == 0, nums))
print("evens:", evens)


## 5) filter() — TRICKY: filtering out None / empty strings

**Very common** in real data cleaning.

### Trick 1: `filter(None, iterable)`
If you pass `None` as the function, filter removes **falsy** values:
- None
- 0
- ''
- []
- {}
- False


In [ ]:
data = ["orders", "", None, "customers", "   ", 0, "products"]

# removes falsy values: "" and None and 0, but note: "   " is NOT falsy
filtered_falsy = list(filter(None, data))
print("filtered_falsy:", filtered_falsy)

# remove empty/whitespace strings properly
filtered_clean = list(filter(lambda s: s is not None and str(s).strip() != "", data))
print("filtered_clean:", filtered_clean)


## 6) filter() — Multiple conditions (AND/OR) (TRICKY)

Example: keep numbers that are even AND not equal to 6.


In [ ]:
nums = [2, 4, 6, 8, 10]
res = list(filter(lambda x: (x % 2 == 0) and (x != 6), nums))
print(res)


## 7) reduce() — Basics

### Pattern
```python
reduce(function, iterable, initializer?)
```

reduce combines items cumulatively to a **single value**.

Example: sum
```python
reduce(lambda acc, x: acc + x, [1,2,3], 0)
```

### TRICKY: initializer matters a LOT.


In [ ]:
nums = [1, 2, 3, 4]
total1 = reduce(lambda acc, x: acc + x, nums)       # no initializer
total2 = reduce(lambda acc, x: acc + x, nums, 0)    # initializer 0

print("reduce without initializer:", total1)
print("reduce with initializer:", total2)


## 8) reduce() — TRICKY: empty list behavior

- If iterable is empty and **no initializer**: ❌ error
- If iterable is empty and **initializer exists**: ✅ returns initializer


In [ ]:
empty = []

try:
    print(reduce(lambda acc, x: acc + x, empty))
except TypeError as e:
    print("Error without initializer:", e)

print("With initializer:", reduce(lambda acc, x: acc + x, empty, 0))


## 9) reduce() — Product of numbers (classic)

Trick: initializer should be **1** for product.


In [ ]:
nums = [2, 3, 4]
product = reduce(lambda acc, x: acc * x, nums, 1)
print("product:", product)


## 10) reduce() — max / min (TRICKY)

You can compute max/min using reduce.

Trick: handle empty lists with initializer or guard.


In [ ]:
nums = [10, 3, 22, 7]
max_val = reduce(lambda a, b: a if a > b else b, nums)
min_val = reduce(lambda a, b: a if a < b else b, nums)
print("max:", max_val)
print("min:", min_val)


## 11) reduce() — Flatten a list of lists (very common)

This is similar to your nested-list flattening example.

```python
reduce(lambda acc, sub: acc + sub, nested, [])
```

Trick: initializer must be `[]`.


In [ ]:
nested = [[1, 2], [3, 4], [5]]
flat = reduce(lambda acc, sub: acc + sub, nested, [])
print("flat:", flat)


## 12) reduce() — Join strings safely (TRICKY)

Reduce can join strings, but Python has a better built-in: `"".join(list)`.

Still, interviewers sometimes ask reduce version.

Trick: initializer should be "".


In [ ]:
words = ["data", "engineer", "rocks"]
joined = reduce(lambda acc, w: acc + " " + w, words, "").strip()
print(joined)

better = " ".join(words)
print("better:", better)


## 13) reduce() — Merge dictionaries (TRICKY + useful)

Suppose you have many small dictionaries and want one merged.

Trick: use initializer `{}`.

Note: if keys overlap, later dict overwrites earlier.


In [ ]:
dicts = [
    {"a": 1, "b": 2},
    {"b": 20, "c": 3},
    {"d": 4}
]

merged = reduce(lambda acc, d: {**acc, **d}, dicts, {})
print("merged:", merged)


## 14) reduce() — Frequency count (TRICKY but common)

Build a frequency dictionary using reduce.

Trick: initializer must be `{}`.


In [ ]:
items = ["orders", "orders", "products", "orders", "customers", "products"]

freq = reduce(
    lambda acc, x: {**acc, x: acc.get(x, 0) + 1},
    items,
    {}
)
print("freq:", freq)


## 15) Combined mini practice (Map + Filter + Reduce)

### Problem:
Given a list of strings representing numbers (some invalid),
1) keep only strings that are valid integers
2) convert to int
3) keep only even numbers
4) return the sum of those even numbers

This is a realistic pipeline style question.


In [ ]:
raw = ["10", " 20", "x", "30", "-4", "5.5", "0", "  "]

def is_int_string(s):
    if s is None:
        return False
    s = str(s).strip()
    if s == "":
        return False
    if s[0] in "+-":
        return s[1:].isdigit() and len(s) > 1
    return s.isdigit()

valid = list(filter(is_int_string, raw))
nums = list(map(lambda s: int(str(s).strip()), valid))
evens = list(filter(lambda x: x % 2 == 0, nums))
total_even = reduce(lambda acc, x: acc + x, evens, 0)

print("raw:", raw)
print("valid strings:", valid)
print("nums:", nums)
print("evens:", evens)
print("sum(evens):", total_even)


## 16) Quick Tricky Q&A (Interview-style)

### Q1) Why does `map(...)` print as `<map object ...>`?
- Because it returns an iterator (lazy). Convert to list or iterate.

### Q2) Why does `list(m)` work once and then becomes empty?
- Because iterators are exhausted after one pass.

### Q3) What happens if reduce is used on empty list without initializer?
- TypeError.

### Q4) Why do we use initializer in reduce?
- Prevent empty-list errors and choose correct identity values (`0` for sum, `1` for product, `[]` for list concat, `{}` for dict merge).

### Q5) What does `filter(None, data)` do?
- Removes falsy values (None, 0, '', False, empty collections).


---
## ✅ Recap

- `map()` transforms every item.
- `filter()` selects items that satisfy a condition.
- `reduce()` collapses an iterable into one value.
- All three return iterators (convert to list if needed).
- `reduce(..., initializer)` is crucial for correctness and edge cases.
